In [1]:
cd E:\PycharmProjects\IIRS_Intern\17

E:\PycharmProjects\IIRS_Intern\17


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from keras.layers import Dense, Conv1D, Input, MaxPooling1D, Flatten, Dropout, LSTM
from keras import Sequential
from keras.utils import np_utils
from PIL import Image
import array
import sys
import os

gdal.GetDriverByName('EHdr').Register()
img = gdal.Open("apex17bnads")
b = img.RasterCount
col = img.RasterXSize
row = img.RasterYSize
bands = img.RasterCount
datatype = gdal.GetDataTypeName(bands)
#c_c = int(input("Enter Number of Classes"))
c_c = 8
print(datatype)
def train():
    def ReadBilFile(bil,bands,pixels):
        extract_band = 1
        image = np.zeros([pixels, bands], dtype=np.uint16)
        gdal.GetDriverByName('EHdr').Register()
        img = gdal.Open(bil)
        while bands >= extract_band:
            bandx = img.GetRasterBand(extract_band)
            datax = bandx.ReadAsArray()
            temp = datax
            store = temp.reshape(pixels)
            for i in range(pixels):
                image[i][extract_band - 1] = store[i]
            extract_band = extract_band + 1
        return image

    print(os.listdir("E:\\PycharmProjects\\IIRS_Intern\\17"))
    pixels = row * col
    y_test = np.zeros([row * col], dtype=np.uint16)
    x_test = ReadBilFile("apex17bnads", 17, pixels)
    x_test = x_test.reshape(row*col, 17, 1)
    values = []
    path = ["cforest17", "dforest17", "grass17", "river17bands", "art_truf", "roads17", "buildings17"]
    dict1 = {"cforest17":0, "dforest17":0, "grass17":0, "river17bands":0, "art_truf":0, "roads17":0, "buildings17":0}
    c_l = {"cforest17":1, "dforest17":2, "grass17":3, "river17bands":4, "art_truf":5, "roads17":6, "buildings17":7}
    clicks={}
    for address in path:
        with open(address, "rb") as f:
            k = len(f.read())
            clicks[address] = (k // 2 // bands) if (k // 2 // bands) < 400 else (k // 2 // bands) // 4
            print('{} ==> {}'.format(address, clicks[address]))
             
    for address in path:
        with open(address, "rb") as f:
            b = array.array("H")
            b.fromfile(f, clicks[address]*bands)
            if sys.byteorder == "little":
                b.byteswap()
            for v in b:
                values.append(v)

    ll = (len(values))
    rex = ll // bands
    print(ll, rex)
    '''from here'''
    f_in = np.zeros([ll], dtype=np.uint16)
    x = 0
    for i in range(ll):
        f_in[x] = values[i]
        x += 1

    sh = int(rex // bands)
    y_train = np.zeros([rex], dtype=np.uint16)
    
    """    print(
        (dict1["Clay"] + dict1["Dalforest"] + dict1["Eufinal"] + dict1["Water"] + dict1["Wheat"] + dict1["Grassland"]) // 2)
    for i in range(dict1["Clay"] // 8):
        y_train[i] = 1
    for i in range(dict1["Dalforest"] // 8):
        y_train[dict1["Clay"] // 8 + i] = 2
    for i in range(dict1["Eufinal"] // 8):
        y_train[(dict1["Clay"] + dict1["Dalforest"]) // 8 + i] = 3
    for i in range(dict1["Water"] // 8):
        y_train[(dict1["Clay"] + dict1["Dalforest"] + dict1["Eufinal"]) // 8 + i] = 4
    for i in range(dict1["Wheat"] // 8):
        y_train[(dict1["Clay"] + dict1["Dalforest"] + dict1["Eufinal"] + dict1["Water"]) // 8 + i] = 5
    for i in range(dict1["Grassland"] // 8):
        y_train[(dict1["Clay"] + dict1["Dalforest"] + dict1["Eufinal"] + dict1["Water"] + dict1["Wheat"]) // 8 + i] = 6"""
   
    '''
    till here
    '''
    mark = 0
    for add in path:
        for i in range(clicks[add]):
            y_train[mark+i] = c_l[add]
        mark = mark + clicks[add]

        
    x_train = f_in.reshape(rex, bands)

    seed = 7
    np.random.seed(seed)

    x_train = x_train / 2**16-1
    x_test = x_test / 2**16-1
    num_pixels = bands

    for v in y_train:
        print(v, end=" ")

    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    num_classes = c_c
    y_test_new = np.zeros([pixels, c_c], dtype=np.uint16)

    print(x_test)
    print(20*'#')
    print(x_train)
    print(20*'#')
    print(y_test)
    print(20*'#')
    print(y_train)

    print(x_test.shape)
    print(x_train.shape)
    print(y_train.shape)
    print(y_test.shape)

    X = x_train.reshape(x_train.shape[0], 17, 1)

    model = Sequential()
    #model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Conv1D(2 ** 4, 2, activation="relu", padding='same', input_shape=[17, 1]))
    model.add(LSTM(2**5, return_sequences=True))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(2 ** 6, 2, activation="relu", padding='same'))
    model.add(LSTM(2**7, return_sequences=True))
    model.add(MaxPooling1D(2))
    #model.add(Conv1D(2 ** 7, 2, activation="relu", padding='same'))
    #model.add(LSTM(2**8, return_sequences=True))
    #model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Dropout(0.1))
    model.add(Dense(num_classes, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    model.fit(X, y_train, batch_size=50, epochs=3000)
    #model.fit(x_train, y_train, batch_size=10, epochs=1000, verbose=2)
    y_test_new = model.predict(x_test, batch_size=50)
    print(20*'%')
    #print(y_test_new[:,1])
    print(y_test_new.shape)
    #print(np.squeeze(y_test_new))
    print(20*'%')
    y_test1 = np.argmax(y_test_new, axis=1)
    print(30*'*')
    print("this is predicted output")
    #img = x_test.reshape(row, col, bands)
    #plt.imshow(img)
    #plt.show()
    #result = Image.fromarray((img * 255).astype('uint8'))
    #result.save('image.tiff')


    k = y_test1.reshape(row, col)
    plt.imshow(k)
    plt.show()
    result = Image.fromarray((k * (2**16-1)).astype('uint16'))
    result.save('Classified_images_12/hard.tiff')
    
    try:
        os.mkdir("Classified_images_12")
    except:
        pass
    
    for i in range(1, 8):
        img = y_test_new[:,i].reshape(row, col)
        plt.imshow(img*(2**16-1))
        plt.colorbar()
        plt.show()
        result = Image.fromarray(((img * (2**16-1))).astype('uint16'))
        result.save('Classified_images_12/'+str(i)+'_other.tiff')


train()



C:\Users\HP\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


None
['apex17bnads', 'apex17bnads.hdr', 'art_truf', 'Band17DullForest', 'Bright_Forest_17_Band', 'buidlings_17_2', 'building', 'buildings17', 'buildings_17_1', 'cforest17', 'Classified_images_12', 'dforest17', 'grass17', 'grass_17', 'river17', 'river17bands', 'river_17', 'roads17', 'road_17', 'soil17']
cforest17 ==> 416
dforest17 ==> 417
grass17 ==> 308
river17bands ==> 417
art_truf ==> 135
roads17 ==> 365
buildings17 ==> 27
35445 2085
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

2085/2085 [==============================] - 2s 793us/step - loss: 1.7903 - acc: 0.1942
Epoch 11/5000
2085/2085 [==============================] - 2s 785us/step - loss: 1.7887 - acc: 0.1866
Epoch 12/5000
2085/2085 [==============================] - 2s 808us/step - loss: 1.7892 - acc: 0.2034
Epoch 13/5000
2085/2085 [==============================] - 2s 731us/step - loss: 1.7890 - acc: 0.1890
Epoch 14/5000
2085/2085 [==============================] - 2s 779us/step - loss: 1.7883 - acc: 0.2000
Epoch 15/5000
2085/2085 [==============================] - 2s 733us/step - loss: 1.7870 - acc: 0.2048
Epoch 16/5000
2085/2085 [==============================] - 2s 736us/step - loss: 1.7877 - acc: 0.1995
Epoch 17/5000
2085/2085 [==============================] - 2s 762us/step - loss: 1.7879 - acc: 0.2029
Epoch 18/5000
2085/2085 [==============================] - 1s 714us/step - loss: 1.7871 - acc: 0.2034
Epoch 19/5000
2085/2085 [==============================] - 1s 694us/step - loss: 1.7870 - acc: 0

2085/2085 [==============================] - 2s 767us/step - loss: 0.5174 - acc: 0.7607
Epoch 168/5000
2085/2085 [==============================] - 2s 820us/step - loss: 0.3994 - acc: 0.8197
Epoch 169/5000
2085/2085 [==============================] - 1s 556us/step - loss: 0.5761 - acc: 0.7372
Epoch 170/5000
2085/2085 [==============================] - 1s 448us/step - loss: 0.5041 - acc: 0.7496
Epoch 171/5000
2085/2085 [==============================] - 1s 609us/step - loss: 0.4816 - acc: 0.7707
Epoch 172/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.4567 - acc: 0.7909
Epoch 173/5000
2085/2085 [==============================] - 2s 814us/step - loss: 0.4034 - acc: 0.8187
Epoch 174/5000
2085/2085 [==============================] - 2s 737us/step - loss: 0.4204 - acc: 0.7990
Epoch 175/5000
2085/2085 [==============================] - 2s 744us/step - loss: 0.4179 - acc: 0.8077
Epoch 176/5000
2085/2085 [==============================] - 2s 738us/step - loss: 0.4588 -

2085/2085 [==============================] - 2s 1ms/step - loss: 0.4333 - acc: 0.7976
Epoch 325/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.3765 - acc: 0.8201
Epoch 326/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.4024 - acc: 0.8177
Epoch 327/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.4028 - acc: 0.8153
Epoch 328/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.3883 - acc: 0.8206
Epoch 329/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.4393 - acc: 0.8062
Epoch 330/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.4203 - acc: 0.8019
Epoch 331/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.5226 - acc: 0.7664
Epoch 332/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.3919 - acc: 0.8288
Epoch 333/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.3708 - acc: 0.8225
Epoch

...

2085/2085 [==============================] - 2s 1ms/step - loss: 0.2131 - acc: 0.9050
Epoch 2526/5000
2085/2085 [==============================] - 2s 947us/step - loss: 0.2666 - acc: 0.8763
Epoch 2527/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2512 - acc: 0.8868
Epoch 2528/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2148 - acc: 0.9050
Epoch 2529/5000
2085/2085 [==============================] - 2s 978us/step - loss: 0.2619 - acc: 0.8758
Epoch 2530/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2003 - acc: 0.9127
Epoch 2531/5000
2085/2085 [==============================] - 2s 974us/step - loss: 0.2134 - acc: 0.9007
Epoch 2532/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2013 - acc: 0.9103
Epoch 2533/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.1923 - acc: 0.9108
Epoch 2534/5000
2085/2085 [==============================] - 2s 960us/step - loss: 0.2062 - 

2085/2085 [==============================] - 2s 959us/step - loss: 0.2791 - acc: 0.8667
Epoch 2683/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2811 - acc: 0.8695
Epoch 2684/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2447 - acc: 0.8959
Epoch 2685/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2365 - acc: 0.8878
Epoch 2686/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2965 - acc: 0.8719
Epoch 2687/5000
2085/2085 [==============================] - 2s 988us/step - loss: 0.2163 - acc: 0.9065 0s - loss: 0.2206 - acc: 0.
Epoch 2688/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2127 - acc: 0.9007
Epoch 2689/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2018 - acc: 0.9127
Epoch 2690/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.1937 - acc: 0.9127
Epoch 2691/5000
2085/2085 [==============================] - 2s 1m

2085/2085 [==============================] - 2s 1ms/step - loss: 0.2158 - acc: 0.9060
Epoch 2761/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2704 - acc: 0.8782
Epoch 2762/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.1999 - acc: 0.9094
Epoch 2763/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2442 - acc: 0.8926
Epoch 2764/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2140 - acc: 0.9074
Epoch 2765/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2417 - acc: 0.8935
Epoch 2766/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2466 - acc: 0.8892
Epoch 2767/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.1928 - acc: 0.9156
Epoch 2768/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2813 - acc: 0.8863
Epoch 2769/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2231 - acc: 0.8

2085/2085 [==============================] - 2s 989us/step - loss: 0.1926 - acc: 0.9156
Epoch 2840/5000
2085/2085 [==============================] - 2s 962us/step - loss: 0.1833 - acc: 0.9189
Epoch 2841/5000
2085/2085 [==============================] - 2s 979us/step - loss: 0.3129 - acc: 0.8590
Epoch 2842/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2825 - acc: 0.8782
Epoch 2843/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2599 - acc: 0.8820
Epoch 2844/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2347 - acc: 0.9002
Epoch 2845/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2404 - acc: 0.8935
Epoch 2846/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2579 - acc: 0.8863
Epoch 2847/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2337 - acc: 0.8969
Epoch 2848/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2424 - ac

2085/2085 [==============================] - 2s 990us/step - loss: 0.1974 - acc: 0.9122
Epoch 2998/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.1806 - acc: 0.9175
Epoch 2999/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2331 - acc: 0.8959
Epoch 3000/5000
2085/2085 [==============================] - 2s 995us/step - loss: 0.3044 - acc: 0.8647
Epoch 3001/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.3175 - acc: 0.8508
Epoch 3002/5000
2085/2085 [==============================] - 2s 945us/step - loss: 0.2194 - acc: 0.9002
Epoch 3003/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2344 - acc: 0.8902A: 0s - loss: 0.2104 - a
Epoch 3004/5000
2085/2085 [==============================] - 2s 1ms/step - loss: 0.2404 - acc: 0.8930
Epoch 3005/5000
2085/2085 [==============================] - 2s 955us/step - loss: 0.2752 - acc: 0.8767
Epoch 3006/5000
2085/2085 [==============================] - 2s 1m